In [1]:
%load_ext lab_black
%cd ..

/mnt/h/hev/log-analytics


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    BertConfig,
)
from torch.optim import AdamW
import torch.nn.functional as F
from tqdm import tqdm
import sys
import re
from collections import defaultdict

In [3]:
def first_word(text, deli=" "):
    for i, t in enumerate(text):
        if t == deli:
            break
    return text[:i]

In [7]:
train = pd.read_csv("data/ori/test.csv")

In [8]:
train

,id,full_log
0,1000000,"Feb 8 15:47:26 localhost kibana: {""type"":""err..."
1,1000001,"Sep 24 03:46:39 localhost kibana: {""type"":""err..."
2,1000002,type=SYSCALL msg=audit(1611888200.428:210563):...
3,1000003,"Jan 18 11:24:06 localhost kibana: {""type"":""err..."
4,1000004,type=SYSCALL msg=audit(1603081202.050:46851): ...
...,...,...
1418911,2418911,"Jan 13 05:07:11 localhost kibana: {""type"":""err..."
1418912,2418912,"Jan 5 02:24:50 localhost kibana: {""type"":""log..."
1418913,2418913,type=SYSCALL msg=audit(1611884593.462:38222): ...
1418914,2418914,Jan 22 01:38:19 localhost logstash: [2021-01-2...


In [10]:
seasons = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

### 날짜 제거?

만약에 날짜 정보를 제거하면 첫 단어가 어떻게 될지?

In [13]:
def remove_pattern(pattern, full_log):
    for s in re.finditer(pattern, full_log):
        a, b = s.span()
        full_log = (full_log[:a] + full_log[b:]).strip()
    return full_log

In [14]:
def remove_date(full_log):
    t = first_word(full_log)
    if len(t) == 4 and t.isdigit() and t[:2] in ("19", "20", "21"):
        full_log = full_log[5:].strip()

    t = first_word(full_log)
    if len(t) == 3 and t in seasons:
        full_log = full_log[4:].strip()

        t = first_word(full_log)
        if t.isdigit():
            full_log = full_log[len(t) + 1 :].strip()

    # 00:00:00 형식의 시간 이면?
    if re.match(r"\d{2}:\d{2}:\d{2}", full_log):
        full_log = full_log[9:].strip()

    if full_log.startswith("localhost"):
        full_log = full_log[10:].strip()

    # sshd[pid] 에서 pid 제거
    # t = first_word(full_log)
    # if re.match(r"[\w\d]+\[\d+\]", t):
    #    u = first_word(t, deli="[")
    #    full_log = (u + " " + full_log[len(t) + 1 :]).strip()

    # @timestamp: "~~~~Z"
    full_log = remove_pattern(r'"@timestamp"\s?:\s?"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z?",?', full_log)
    # "pid": "4567"
    full_log = remove_pattern(r'"pid"\s?:\s?\d+,?', full_log)
    # [pid]
    full_log = remove_pattern(r"\[\d+\]", full_log)

    full_log = full_log.replace("\n", " ")
    full_log = full_log.replace("\r", " ")
    full_log = re.sub(r"\s+", " ", full_log)

    return full_log

In [16]:
first_words = set()
for row, (id, full_log) in tqdm(train.iterrows(), total=len(train), ncols=100, file=sys.stdout):
    full_log = remove_date(full_log)
    t = first_word(full_log)
    first_words.add(t)
    # break

100%|██████████████████████████████████████████████████| 1418916/1418916 [02:07<00:00, 11091.51it/s]


In [17]:
first_words

{'--MARK--:',
 'E:',
 'File',
 'NTFS',
 'OpenSCAP',
 'System',
 'The',
 'Trojaned',
 'WinEvtLog:',
 'Windows',
 'auditd:',
 'augenrules:',
 'com.redhat.imsettings:',
 'dbus-daemon:',
 'dbus:',
 'elasticsearch:',
 'env:',
 'error:',
 'esild-ml-start.sh:',
 'firefox.desktop:',
 'gdm-password]:',
 'gnome-session-binary:',
 'groupadd:',
 'imsettings-start.desktop:',
 'journal:',
 'junipe',
 'kdumpctl:',
 'kernel:',
 'kibana:',
 'level',
 'libcanberra-login-sound.desktop:',
 'libvirtd:',
 'logstash:',
 'm2datateksolaris',
 'mcelog:',
 'node:',
 'oscap:',
 'ossec:',
 'polkitd:',
 'postfix/master:',
 'pulseaudio:',
 'rc.local:',
 'reload_microcode:',
 'sshd:',
 'su:',
 'sudo:',
 'suricata:',
 'sv260',
 'systemd:',
 'type=ANOM_ABEND',
 'type=ANOM_PROMISCUOUS',
 'type=AVC',
 'type=SYSCALL',
 'type=USER_AVC',
 'unix_chkpwd:',
 'useradd:',
 'userdel:',
 'yum:'}

"E:"는 아마도 ERROR 라는 뜻인듯?

logstach, kibana 관련 오류는 수는 많은데 전부 0임.

sshd가 가장 다양하게 갖고있고, --> 레벨별로 메세지 정리해서 확인해보기

System, Windows 같은거는 1정도를 갖는듯?

되도록 학습으로 처리해달라고 한 이유는, kibana, logstash라도 1 이상의 상활이 혹시 발생할지 모르기 때문이 아닐지?
그러면 최종 결과에 weight를 주는 방법으로?

In [204]:
i = 0

In [218]:
print(train.level[i], remove_date(train.full_log[i]))
i += 1

1 type=SYSCALL msg=audit(1611889244.855:247124): arch=c000003e syscall=2 success=yes exit=3 a0=7f1c14d535a4 a1=80000 a2=1 a3=7f1c14f594f8 items=1 ppid=100038 pid=100039 auid=4294967295 uid=0 gid=980 euid=0 suid=0 fsuid=0 egid=980 sgid=980 fsgid=980 tty=(none) ses=4294967295 comm="ps" exe="/usr/bin/ps" subj=system_u:system_r:unconfined_service_t:s0 key="audit-wazuh-r" type=CWD msg=audit(1611889244.855:247124): cwd="/" type=PATH msg=audit(1611889244.855:247124): item=0 name="/etc/ld.so.cache" inode=35395307 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0 type=PROCTITLE msg=audit(1611889244.855:247124): proctitle=2F62696E2F7073002D700032353336
